#0) Imports and Installs

In [ ]:
!pip install yfinance

In [ ]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

import requests


# Question 1: IPO Filings Web Scraping and Data Processing

## What's the total sum ($m) of 2023 filings that happenned of Fridays?

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/ Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs). Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given). You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string. Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all fillings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 24 of it is not null.

(additional: you can read about S-1 IPO filing to understand the context)


In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [ ]:
ipo_df = ipo_dfs[0]

In [ ]:
# Convert the 'Filing Date' to datetime()
ipo_df['Filing Date'] = pd.to_datetime(ipo_df['Filing Date'])
ipo_df

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000
1,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
2,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000
3,2024-04-26,HDL,Super Hi International Holding Ltd.,-,-
4,2024-04-22,DRJT,Derun Group Inc,$5.00,-
...,...,...,...,...,...
322,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000
323,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000
324,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000
325,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000


In [ ]:
# Convert 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs)
ipo_df['Shares Offered'] = ipo_df['Shares Offered'].replace('-', np.NaN).astype('float64')

In [ ]:
ipo_df.dtypes

Filing Date       datetime64[ns]
Symbol                    object
Company Name              object
Price Range               object
Shares Offered           float64
dtype: object

In [ ]:
# Define a new field 'Avg_price' based on the "Price Range",
# which equals to NaN if no price is specified,
# to the price (if only one number is provided),
# or to the average of 2 prices (if a range is given)

def calc_avg_price(price_range):
    if price_range == '-':
        return np.NaN
    elif price_range.find(' - ') == -1:
        return float(price_range[price_range.find('$')+1:])
    else:
        first_num = float(price_range[price_range.find('$')+1:price_range.find(' ')])
        second_num = float(price_range[price_range.rfind('$') + 1:])
        return (first_num + second_num) / 2

In [ ]:
ipo_df['Avg_price'] = ipo_df['Price Range'].apply(lambda x: calc_avg_price(x))
ipo_df['Avg_price']

0       3.00
1      10.00
2      10.00
3        NaN
4       5.00
       ...  
322     9.00
323    10.00
324     9.00
325     4.00
326     5.75
Name: Avg_price, Length: 327, dtype: float64

In [ ]:
# Define a column "Shares_offered_value",
# which equals to "Shares Offered" * "Avg_price"
# (when both columns are defined; otherwise, it's NaN)

ipo_df['Shares_offered_value'] = ipo_df['Shares Offered'] * ipo_df['Avg_price']
ipo_df['Shares_offered_value']

0      11250000.0
1      60000000.0
2      50000000.0
3             NaN
4             NaN
          ...    
322    13500000.0
323    50000000.0
324    22500000.0
325    17200000.0
326    11500000.0
Name: Shares_offered_value, Length: 327, dtype: float64

In [ ]:
# Find the total sum in $m (millions of USD, closest INTEGER number)
# for all fillings during 2023, which happened on Fridays (Date.dt.dayofweek()==4).
# You should see 32 records in total, 24 of it is not null.

In [ ]:
ipo_df_2023 = ipo_df[ipo_df['Filing Date'].dt.year == 2023]

In [ ]:
ipo_df_2023_fri = ipo_df[(ipo_df['Filing Date'].dt.year == 2023) & (ipo_df['Filing Date'].dt.day_of_week == 4)]

In [ ]:
ipo_df_2023_fri.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 49 to 166
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           32 non-null     datetime64[ns]
 1   Symbol                32 non-null     object        
 2   Company Name          32 non-null     object        
 3   Price Range           32 non-null     object        
 4   Shares Offered        25 non-null     float64       
 5   Avg_price             25 non-null     float64       
 6   Shares_offered_value  25 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 2.0+ KB


In [ ]:
ipo_df_2023_fri.Shares_offered_value.sum() / 1000000

285.7

# Question 2: IPOs "Fixed days hold" strategy

## Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?

Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Some of the tickers like 'DYCQ' and 'LEGT' were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day). Find X, when the 75% quantile growth (among 185 investments) is the highest.

In [ ]:
# retrieve the list of IPOs from 2023 and 2024
# from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [ ]:
url = "https://stockanalysis.com/ipos/2024/"

response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      64 non-null     object
 1   Symbol        64 non-null     object
 2   Company Name  64 non-null     object
 3   IPO Price     64 non-null     object
 4   Current       64 non-null     object
 5   Return        64 non-null     object
dtypes: object(6)
memory usage: 3.1+ KB


In [ ]:
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)
stacked_ipos_df.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$28.91,12.46%
1,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.23,-23.00%
2,"Apr 25, 2024",RBRK,"Rubrik, Inc.",$32.00,$32.85,5.97%
3,"Apr 25, 2024",LOAR,Loar Holdings Inc.,$28.00,$49.00,68.93%
4,"Apr 25, 2024",MRX,Marex Group plc,$19.00,$18.97,-2.47%


In [ ]:
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format='%b %d, %Y')

In [ ]:
# Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01")
# Remove 'RYZB', as it is no longer available on Yahoo Finance.

stacked_ipos_df[stacked_ipos_df['IPO Date'] < '2024-03-01']['Symbol']

33     SMXT
34     VHAI
35     DYCQ
36     CHRO
37     UMAC
       ... 
213     QSG
214    CVKD
215    SKWD
216    ISRL
217    MGOL
Name: Symbol, Length: 185, dtype: object

In [ ]:
to_download = stacked_ipos_df[stacked_ipos_df['IPO Date'] < '2024-03-01']['Symbol'].to_list()
to_download

['SMXT',
 'VHAI',
 'DYCQ',
 'CHRO',
 'UMAC',
 'TBBB',
 'HLXB',
 'MGX',
 'TELO',
 'KYTX',
 'PMNT',
 'AHR',
 'LEGT',
 'ANRO',
 'GUTS',
 'AS',
 'FBLG',
 'BTSG',
 'AVBP',
 'HAO',
 'CGON',
 'YIBO',
 'SUGP',
 'JL',
 'KSPI',
 'JVSA',
 'PSBD',
 'CCTG',
 'SYNX',
 'SDHC',
 'ROMA',
 'IROH',
 'LGCB',
 'ZKH',
 'BAYA',
 'INHD',
 'AFJK',
 'GSIW',
 'FEBO',
 'CLBR',
 'ELAB',
 'RR',
 'DDC',
 'SHIM',
 'GLAC',
 'SGN',
 'HG',
 'CRGX',
 'ANSC',
 'AITR',
 'GVH',
 'LXEO',
 'PAPL',
 'ATGL',
 'MNR',
 'WBUY',
 'NCL',
 'BIRK',
 'GMM',
 'PMEC',
 'LRHC',
 'GPAK',
 'SPKL',
 'QETA',
 'MSS',
 'ANL',
 'SYRA',
 'VSME',
 'LRE',
 'TURB',
 'MDBH',
 'KVYO',
 'CART',
 'DTCK',
 'RYZB',
 'NMRA',
 'ARM',
 'SPPL',
 'NWGL',
 'SWIN',
 'IVP',
 'NNAG',
 'SRM',
 'SPGC',
 'LQR',
 'NRXS',
 'FTEL',
 'MIRA',
 'PXDT',
 'CTNT',
 'HRYU',
 'SRFM',
 'PRZO',
 'HYAC',
 'KVAC',
 'JNVR',
 'ELWS',
 'WRNT',
 'TSBX',
 'ODD',
 'APGE',
 'NETD',
 'SGMT',
 'BOWN',
 'SXTP',
 'PWM',
 'VTMX',
 'INTS',
 'SVV',
 'KGS',
 'FIHL',
 'GENK',
 'BUJA',
 'BOF',
 'AZ

In [ ]:
to_download.remove('RYZB')

In [ ]:
to_download.remove('PTHR')
# to_download.append('IBACU')

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i, ticker in enumerate(to_download):

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

  historyPrices['Ticker'] = ticker
  historyPrices['Year'] = historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in range(1, 31):
    historyPrices['future_'+str(i)+'d'] = historyPrices['Adj Close'].shift(-i) / historyPrices['Adj Close']
#   historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)

  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [ ]:
stocks_df = stocks_df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1)
stocks_df.tail()

,Adj Close,Ticker,Year,Month,Weekday,Date,future_1d,future_2d,future_3d,future_4d,...,future_21d,future_22d,future_23d,future_24d,future_25d,future_26d,future_27d,future_28d,future_29d,future_30d
37090,0.240,MGOL,2024,4,0,2024-04-29,0.983333,1.041667,0.970833,0.966667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37091,0.236,MGOL,2024,4,1,2024-04-30,1.059322,0.987288,0.983051,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37092,0.250,MGOL,2024,5,2,2024-05-01,0.932000,0.928000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37093,0.233,MGOL,2024,5,3,2024-05-02,0.995708,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37094,0.232,MGOL,2024,5,4,2024-05-03,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
min_dates = stocks_df.groupby(['Ticker']).Date.agg(['min'])
min_dates

,min
Ticker,
AACT,2023-06-13
AESI,2023-03-09
AFJK,2024-01-23
AHR,2024-02-07
AITR,2024-01-02
...,...
WRNT,2023-07-25
YGFGF,2023-03-28
YIBO,2024-01-25


In [ ]:
merged = min_dates.merge(stocks_df, how='inner', left_on = ['Ticker', 'min'], right_on=['Ticker', 'Date'])
merged

,Ticker,min,Adj Close,Year,Month,Weekday,Date,future_1d,future_2d,future_3d,...,future_21d,future_22d,future_23d,future_24d,future_25d,future_26d,future_27d,future_28d,future_29d,future_30d
0,AACT,2023-06-13,10.110000,2023,6,1,2023-06-13,0.999011,1.000000,1.000000,...,1.005539,1.004946,1.004946,1.004946,1.005935,1.005935,1.005935,1.007913,1.007913,1.007913
1,AESI,2023-03-09,16.125948,2023,3,3,2023-03-09,0.973451,0.943363,0.979941,...,1.023009,1.049558,1.050737,1.058997,1.076106,1.074926,1.057817,1.066077,1.056637,1.072566
2,AFJK,2024-01-23,10.160000,2024,1,1,2024-01-23,1.000000,1.000984,1.000984,...,1.001969,1.001476,1.001969,1.001378,1.001969,0.999409,1.001476,1.002953,1.001969,1.002362
3,AHR,2024-02-07,12.991912,2024,2,2,2024-02-07,0.987897,0.992436,0.994705,...,1.044629,1.047655,1.045386,1.040847,1.043873,1.027988,1.046142,1.051437,1.062027,1.072617
4,AITR,2024-01-02,10.140000,2024,1,1,2024-01-02,1.000986,1.003945,1.003945,...,1.006903,1.006903,1.006903,1.007890,1.008876,1.007890,1.007692,1.007692,1.008383,1.008876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,WRNT,2023-07-25,4.300000,2023,7,1,2023-07-25,0.788372,0.572093,0.508139,...,0.241860,0.223256,0.226744,0.209302,0.244186,0.217674,0.209302,0.209302,0.213953,0.225814
179,YGFGF,2023-03-28,3.890000,2023,3,1,2023-03-28,0.975064,0.976864,0.997686,...,0.950386,0.904884,1.002571,0.866324,0.868895,0.750643,0.758355,0.754499,0.830334,0.943445
180,YIBO,2024-01-25,2.790000,2024,1,3,2024-01-25,0.974910,0.931900,0.953405,...,0.931900,0.982079,1.014337,0.982079,0.996416,1.021505,1.114695,1.057348,0.885305,0.913978
181,ZJYL,2023-03-28,0.397500,2023,3,1,2023-03-28,0.997987,0.962264,0.937107,...,0.899371,0.959748,1.207547,0.739623,0.691824,0.704403,0.716981,0.704403,0.716730,0.703145


In [ ]:
future_cols = [x for x in merged.keys() if x.find('future')>=0]
len(future_cols)

30

In [ ]:
merged[future_cols].describe().loc['75%']

future_1d     1.011917
future_2d     1.017923
future_3d     1.009769
future_4d     1.009227
future_5d     1.009648
future_6d     1.007383
future_7d     1.006880
future_8d     1.008356
future_9d     1.011662
future_10d    1.011136
future_11d    1.013322
future_12d    1.016577
future_13d    1.019981
future_14d    1.017457
future_15d    1.018500
future_16d    1.015147
future_17d    1.010853
future_18d    1.014395
future_19d    1.015059
future_20d    1.014601
future_21d    1.013281
future_22d    1.030276
future_23d    1.026547
future_24d    1.029366
future_25d    1.028571
future_26d    1.028524
future_27d    1.039552
future_28d    1.040796
future_29d    1.026364
future_30d    1.024294
Name: 75%, dtype: float64

In [ ]:
merged[future_cols].describe().loc['75%'].max()

1.0407960039433721

# Question 3: Is Growth Concentrated in the Largest Stocks?

## Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?


In [ ]:
start_date = '2013-12-20'
end_date = '2023-12-31'

In [ ]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

len(LARGEST_STOCKS)

33

In [ ]:
NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

len(set(LARGE_STOCKS))

33

In [ ]:
ALL_STOCKS = LARGEST_STOCKS + LARGE_STOCKS

In [ ]:
len(ALL_STOCKS)

66

In [ ]:
stocks_df = pd.DataFrame({'A' : []})

for i, ticker in enumerate(ALL_STOCKS):
  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                              start = start_date,
                              end = end_date,
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year'] = historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  historyPrices['growth_'+str(7)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(7)
#   historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [ ]:
def get_ticker_category(ticker: str, large_list, largest_list):
  if ticker in large_list:
    return 'LARGE'
  elif ticker in largest_list:
    return 'LARGEST'
  else:
    return 'ERROR'

In [ ]:
stocks_df['ticker_category'] = stocks_df.Ticker.apply(lambda x: get_ticker_category(x, LARGE_STOCKS, LARGEST_STOCKS))

In [ ]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [ ]:
grouped = stocks_df[stocks_df.Date>='2014-01-01'].groupby(['Date', 'ticker_category']).growth_7d.agg(['mean']).copy()

In [ ]:
grouped

mean
Date       ticker_category          
2014-01-01 LARGE            1.009288
           LARGEST          1.011797
2014-01-02 LARGE            0.999993
           LARGEST          1.002964
2014-01-03 LARGE            1.002364
...                              ...
2023-12-27 LARGEST          1.015420
2023-12-28 LARGE            0.999755
           LARGEST          1.009344
2023-12-29 LARGE            0.999948
           LARGEST          1.005809

[5190 rows x 1 columns]

In [ ]:
pd.pivot_table(grouped, values='mean', index='Date', columns='ticker_category')

ticker_category,LARGE,LARGEST
Date,,
2014-01-01,1.009288,1.011797
2014-01-02,0.999993,1.002964
2014-01-03,1.002364,0.997167
2014-01-06,0.998084,0.993116
2014-01-07,0.994795,0.992474
...,...,...
2023-12-22,1.003634,1.014285
2023-12-26,1.014441,1.014900
2023-12-27,1.002853,1.015420


In [ ]:
view_1 = pd.pivot_table(grouped, values='mean', index='Date', columns='ticker_category').copy()

In [ ]:
view_1['large_better'] = np.where(view_1['LARGE'] > view_1['LARGEST'], 1, 0)
view_1

ticker_category,LARGE,LARGEST,large_better
Date,,,
2014-01-01,1.009288,1.011797,0
2014-01-02,0.999993,1.002964,0
2014-01-03,1.002364,0.997167,1
2014-01-06,0.998084,0.993116,1
2014-01-07,0.994795,0.992474,1
...,...,...,...
2023-12-22,1.003634,1.014285,0
2023-12-26,1.014441,1.014900,0
2023-12-27,1.002853,1.015420,0


In [ ]:
view_1.large_better.value_counts()

large_better
0    1378
1    1217
Name: count, dtype: int64

In [ ]:
# Calculate the number of days when the LARGE GROUP outperforms the LARGEST GROUP
# divide it by the total number of trading days (which should be 2595 days),
# and convert it to a percentage (closest INTEGER value).
view_1.large_better.value_counts()[1] / 2595 * 100

46.897880539499035

### Without grouping first

In [ ]:
view_2 = pd.pivot_table(stocks_df, values='growth_7d', index='Date', columns='ticker_category', aggfunc='mean')
view_2

ticker_category,LARGE,LARGEST
Date,,
2014-01-01,1.011684,1.011797
2014-01-02,0.997706,1.002964
2014-01-03,0.999945,0.997167
2014-01-06,0.996087,0.993116
2014-01-07,0.991651,0.992473
...,...,...
2023-12-22,1.002868,1.014285
2023-12-26,1.013505,1.014900
2023-12-27,1.001354,1.015420


In [ ]:
view_2['large_better'] = np.where(view_2['LARGE'] > view_2['LARGEST'], 1, 0)
view_2

ticker_category,LARGE,LARGEST,large_better
Date,,,
2014-01-01,1.011684,1.011797,0
2014-01-02,0.997706,1.002964,0
2014-01-03,0.999945,0.997167,1
2014-01-06,0.996087,0.993116,1
2014-01-07,0.991651,0.992473,0
...,...,...,...
2023-12-22,1.002868,1.014285,0
2023-12-26,1.013505,1.014900,0
2023-12-27,1.001354,1.015420,0


In [ ]:
view_2.large_better.value_counts()[1] / 2595 * 100

47.70712909441233

# Question 4: Trying Another Technical Indicators strategy
## What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?

In [1]:
# answer in another file